In [1]:
256-128

128

In [2]:
128+64


192

In [1]:
import os
import random
from PIL import Image
import numpy as np
import pandas as pd
#for image transform
import cv2

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import random
import time

from model_loader_CP2 import *
#from CP_helper import *
from Unet import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box

%load_ext autoreload
%autoreload 2
%load_ext autotime

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

time: 1.93 ms


In [3]:
torch.cuda.is_available()

True

time: 636 ms


In [4]:
image_folder = 'data'
annotation_csv = 'data/annotation.csv'

time: 600 µs


In [5]:
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)

time: 743 µs


In [6]:
labeled_scene_index

array([106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133])

time: 1.9 ms


In [7]:
len(labeled_scene_index)

28

time: 1.52 ms


In [8]:
labeled_scene_index_shuf = labeled_scene_index
random.shuffle(labeled_scene_index_shuf)

time: 657 µs


In [9]:
labeled_scene_index_shuf

array([109, 120, 116, 123, 111, 106, 113, 127, 125, 124, 129, 131, 110,
       108, 128, 112, 117, 126, 115, 132, 121, 122, 114, 107, 119, 130,
       118, 133])

time: 1.77 ms


In [10]:
train_labeled_scene_index = labeled_scene_index_shuf[:-10]
val_labeled_scene_index = labeled_scene_index_shuf[-10: 0 ]

time: 683 µs


In [11]:
train_labeled_scene_index

array([109, 120, 116, 123, 111, 106, 113, 127, 125, 124, 129, 131, 110,
       108, 128, 112, 117, 126])

time: 1.75 ms


In [12]:
kwargs = {
    #'first_dim': 'sample',
    'transform': transforms.ToTensor(),
    'image_folder': image_folder,
    'annotation_file': annotation_csv,
    'extra_info': True}

def gen_train_val_loader(labeled_scene_index, **kwargs):
    labeled_scene_index_shuf = labeled_scene_index
    random.shuffle(labeled_scene_index_shuf)
    train_labeled_scene_index = labeled_scene_index_shuf[:-10] #hard code we know there are only 28 scenes that are labeled
    val_labeled_scene_index = labeled_scene_index_shuf[-10:]
    
    print(len(train_labeled_scene_index), len(val_labeled_scene_index))
    print(train_labeled_scene_index[0], val_labeled_scene_index[0])
        
    loadkwargs = {'batch_size': 2,
    'shuffle': True,
    'collate_fn':collate_fn,
    'num_workers':2,
    
    }
    
    labeled_trainset = LabeledDataset(scene_index=train_labeled_scene_index, **kwargs)
    print(len(labeled_trainset))
    trainloader = torch.utils.data.DataLoader(labeled_trainset, **loadkwargs)
    
    labeled_valset = LabeledDataset(scene_index=val_labeled_scene_index, **kwargs)  
    print(len(labeled_valset))
    valloader = torch.utils.data.DataLoader(labeled_valset, **loadkwargs)
    
    result={"train" : trainloader,
           "val": valloader
        
    }
    return result

time: 2.44 ms


In [13]:
train_val_loader = gen_train_val_loader(labeled_scene_index, **kwargs)

18 10
110 117
2268
1260
time: 255 ms


In [14]:
train_loader = train_val_loader["train"]
for i ,(sample, target, road_image, extra) in enumerate(train_loader):
    print(len(sample))
    print(sample[0].shape)
    print(road_image[0].shape)
    
    break
    

2
torch.Size([6, 3, 256, 306])
torch.Size([800, 800])
time: 279 ms


In [15]:
##spot check
sample_ = ModelLoader.get_binary_road_map(sample).cuda() #should be [batch size, 800, 800]
labels = torch.stack(road_image, 0).cuda() #should be [batch size, 800, 800]

time: 2.15 s


In [16]:
sample_.shape

torch.Size([2, 800, 800])

time: 2.02 ms


In [17]:
labels.int().sum()

tensor(429651, device='cuda:0')

time: 2.42 ms


In [18]:
(sample_>0.5).int().sum()
 

tensor(699921, device='cuda:0')

time: 1.99 ms


In [19]:
(sample_>0.5).int().shape

torch.Size([2, 800, 800])

time: 1.74 ms


In [20]:
labels.int().shape

torch.Size([2, 800, 800])

time: 1.82 ms


In [21]:
(sample_>0.5).int().eq(labels.int()).sum().item()

630356

time: 2.17 ms


In [22]:
630356/1280000

0.492465625

time: 1.52 ms


In [23]:
testUnet = UNet(in_channel=1,out_channel=1).cuda()

time: 75 ms


In [24]:
output1 = testUnet(sample_.unsqueeze(1))

bottleneck1 torch.Size([2, 256, 184, 184]),  encode_block3torch.Size([2, 256, 193, 193])
decode_block3torch.Size([2, 512, 184, 184])
cat_layer2 torch.Size([2, 128, 360, 360]),  decode_block2torch.Size([2, 128, 394, 394])
decode_block2torch.Size([2, 256, 360, 360])
cat_layer1 shape torch.Size([2, 64, 712, 712]), encode_bl1 shapetorch.Size([2, 64, 796, 796])
pre final shape torch.Size([2, 128, 712, 712])
pre refit layer shape torch.Size([2, 1, 708, 708])
time: 1.95 s


In [25]:
output1.shape

torch.Size([2, 1, 800, 800])

time: 1.77 ms


In [26]:
criterion = torch.nn.BCELoss(reduction = "mean")

time: 733 µs


In [27]:
loss = criterion(output1.squeeze(1), labels.float())

time: 25.2 ms


In [28]:
output1.squeeze(1).sum()

tensor(611754.4375, device='cuda:0', grad_fn=<SumBackward0>)

time: 3.32 ms


In [29]:
loss

tensor(0.7723, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

time: 2.33 ms


In [30]:
criterion2 = torch.nn.BCELoss(reduction = "sum")


time: 684 µs


In [31]:
loss2 = criterion(output1.squeeze(1), labels.float())

time: 1.06 ms


In [32]:
loss2

tensor(0.7723, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

time: 2.51 ms


In [33]:
labels.view(1, -1).shape

torch.Size([1, 1280000])

time: 1.9 ms


In [34]:
output1.view(1, -1).shape

torch.Size([1, 1280000])

time: 1.96 ms


In [35]:
output1.max()

tensor(1.0000, device='cuda:0', grad_fn=<MaxBackward1>)

time: 2.49 ms


In [36]:
sjafk;dsa

NameError: name 'sjafk' is not defined

time: 117 ms


In [ ]:
len(param_list)

In [ ]:
train_kwargs={
    'epochs':1,
    "lr": 0.01,
    'momentum': 0.99
    }



def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
     
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for i ,(sample, target, road_image, extra) in enumerate(loader):
             
            sample_ = ModelLoader.get_binary_road_map(sample).cuda() #should be [batch size, 800, 800]
            labels = torch.stack(road_image, 0).cuda() #should be [batch size, 800, 800]
            
            
            outputs = model(sample_.unsqueeze(1))
            outputs = outputs.squeeze(1)
            predicted = (outputs>0.5).int() ## convert to bineary
            
            total += (labels.size(0)*labels.size(1)*labels.size(2))
            correct += predicted.eq(labels.int()).sum().item()
        
    return (100 * correct / total)
     


def train(train_val_loader, **train_kwargs):
    #initialize stuff...
    train_loader = train_val_loader["train"]
    val_loader = train_val_loader["val"]
    
    unet = UNet(in_channel=1,out_channel=1).cuda()
    criterion = torch.nn.BCELoss()
    param_list = [p for p in unet.parameters() if p.requires_grad]
    print(len(praam_list))
    optimizer = torch.optim.SGD(param_list, lr = train_kwargs["lr"], momentum=train_kwargs["momentum"])
    train_losses = []
    val_accs = []
    
    unet.train()
    for e in range(train_kwargs["epochs"]):
        t = time.process_time()

        for i ,(sample, target, road_image, extra) in enumerate(train_loader):
            sample_ = ModelLoader.get_binary_road_map(sample).cuda() #should be [batch size, 800, 800]
            labels = torch.stack(road_image, 0).cuda() #should be [batch size, 800, 800]
            
            optimizer.zero_grad()
            outputs = unet(sample_.unsqueeze(1)) #unet needs the channels dimension
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            
            # validate every 200 iterations
            if i > 0 and i % 100== 0:
                val_acc = test_model(val_loader, unet) #calls model.eval()
                val_accs.append(val_acc)
                #do some stuff
                elapsed_time = time.process_time() - t
                print('Epoch: [{}], Step: [{}], Train Loss {:.4f}, Validation Acc: {:.4f}, time {:.4f}'.format( 
                           e+1, i+1, loss,  val_acc, elapsed_time))
                unet.train() #go back to training
                t = time.process_time()
    

In [ ]:
train_kwargs={
    'epochs':1,
    "lr": 0.01,
    'momentum': 0.99
    }

train(train_val_loader, **train_kwargs)

In [ ]:
sdfafdsafdsafs

In [ ]:
## old code

In [ ]:
#get sample
transform = torchvision.transforms.ToTensor()

unlabeled_trainset = UnlabeledDataset(image_folder=image_folder,scene_index=unlabeled_scene_index, first_dim='sample', transform=transform)
trainloader = torch.utils.data.DataLoader(unlabeled_trainset, batch_size=3, shuffle=True, num_workers=2)

In [ ]:
# [batch_size, 6(images per sample), 3, H, W]
sample = iter(trainloader).next()
print(sample.shape)

In [ ]:
#The 6 images orgenized in the following order:
# 0 CAM_FRONT_LEFT, 1 CAM_FRONT, 2 CAM_FRONT_RIGHT, 3 CAM_BACK_LEFT, 4 CAM_BACK, 5 CAM_BACK_RIGHT
plt.imshow(torchvision.utils.make_grid(sample[2], nrow=3).numpy().transpose(1, 2, 0)) #need the transpose
plt.axis('off');

In [ ]:
# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
LB_trainloader = torch.utils.data.DataLoader(labeled_trainset, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)

In [ ]:
sample, target, road_image, extra = iter(LB_trainloader).next()
print(torch.stack(sample).shape)

In [ ]:
toImg = transforms.ToPILImage()

In [ ]:

toImg(sample[0][5])

In [ ]:
plt.imshow(torchvision.utils.make_grid(sample[0], nrow=3).numpy().transpose(1, 2, 0)) #need the transpose
plt.axis('off');

In [ ]:
## ROAD IMAGE ##
# The road map layout is encoded into a binary array of size [800, 800] per sample 
# Each pixel is 0.1 meter in physiscal space, so 800 * 800 is 80m * 80m centered at the ego car
# The ego car is located in the center of the map (400, 400) and it is always facing the left

fig, ax = plt.subplots()

color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']

ax.imshow(road_image[0], cmap ='binary');

# The ego car position
ax.plot(400, 400, 'x', color="red")

for i, bb in enumerate(target[0]['bounding_box']):
    # You can check the implementation of the draw box to understand how it works 
    draw_box(ax, bb, color=color_list[target[0]['category'][i]])    

In [ ]:
road_image[0].size()

In [ ]:
sample_ = ModelLoader.get_binary_road_map(sample)

In [ ]:
sample_[0].size()

In [ ]:
len(sample_)

In [ ]:
#min max is between 0 and 1
print(sample_[0].min())
print(sample_[1].max())

In [ ]:
import torchvision.models as models
resnet18_pre = models.resnet18(pretrained=True)
resnet18_raw = models.resnet18(pretrained=False)

In [ ]:
#create a normalize tranform 
# res_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [ ]:
#resnet requires all sizes to be 224 by 224
# res_transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])

In [ ]:
comb_img = ModelLoader.sew_images(sample[0])

In [ ]:
comb_img

In [ ]:
# num_ftrs = resnet18_pre.fc.in_features
# resnet18_pre.fc = Identity() #first set it to identity

In [ ]:
##resnet is only for classification

In [ ]:
sample_[0].size()

In [ ]:
sample_.unsqueeze(1).size()

In [ ]:
#afjkd;a

In [ ]:
#try one sample of for Unet
in_channels = 1
out_channels = 1

sing_samp_unet = sample_.unsqueeze(1)
testUnet = UNet(in_channels, out_channels)
testUnetout = testUnet(sing_samp_unet)

In [ ]:
testUnetout.shape

In [ ]:
testUnetout.max()

In [ ]:
torch.stack(road_image[:2],0).unsqueeze(1).int()

In [ ]:
((testUnetout>0.5).int()).shape

In [ ]:
torch.stack(road_image[:2],0).unsqueeze(1).eq((testUnetout>0.5).int()).sum().item()

In [ ]:
649756/(2*800*800)

In [ ]:
#figure out loss Use BCE loss


sample, target, road_image, extra = iter(LB_trainloader).next()

## ROAD IMAGE ##
# The road map layout is encoded into a binary array of size [800, 800] per sample 
# Each pixel is 0.1 meter in physiscal space, so 800 * 800 is 80m * 80m centered at the ego car
# The ego car is located in the center of the map (400, 400) and it is always facing the left

fig, ax = plt.subplots()

color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']

ax.imshow(road_image[0], cmap ='binary');

# The ego car position
ax.plot(400, 400, 'x', color="red")

for i, bb in enumerate(target[0]['bounding_box']):
    # You can check the implementation of the draw box to understand how it works 
    draw_box(ax, bb, color=color_list[target[0]['category'][i]])    


In [ ]:
#figure out loss Use BCE loss


sample, target, road_image, extra = iter(LB_trainloader).next()

## ROAD IMAGE ##
# The road map layout is encoded into a binary array of size [800, 800] per sample 
# Each pixel is 0.1 meter in physiscal space, so 800 * 800 is 80m * 80m centered at the ego car
# The ego car is located in the center of the map (400, 400) and it is always facing the left

fig, ax = plt.subplots()

color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']

ax.imshow(road_image[0], cmap ='binary');

# The ego car position
ax.plot(400, 400, 'x', color="red")

for i, bb in enumerate(target[0]['bounding_box']):
    # You can check the implementation of the draw box to understand how it works 
    draw_box(ax, bb, color=color_list[target[0]['category'][i]])    


In [ ]:


sample, target, road_image, extra = iter(LB_trainloader).next()

## ROAD IMAGE ##
# The road map layout is encoded into a binary array of size [800, 800] per sample 
# Each pixel is 0.1 meter in physiscal space, so 800 * 800 is 80m * 80m centered at the ego car
# The ego car is located in the center of the map (400, 400) and it is always facing the left

fig, ax = plt.subplots()

color_list = ['b', 'g', 'orange', 'c', 'm', 'y', 'k', 'w', 'r']

ax.imshow(road_image[0], cmap ='binary');

# The ego car position
ax.plot(400, 400, 'x', color="red")

for i, bb in enumerate(target[0]['bounding_box']):
    # You can check the implementation of the draw box to understand how it works 
    draw_box(ax, bb, color=color_list[target[0]['category'][i]])    